<a href="https://www.kaggle.com/code/vamsimohanbangale/healthcare?scriptVersionId=251050678" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, FileLink

sns.set(style="whitegrid")

In [ ]:
rf_data = pd.DataFrame({
    'heart_rate': [60, 85, 110, 72, 50, 95],
    'steps': [1000, 8000, 12000, 5000, 2000, 9500],
    'sleep': [5, 8, 6, 7, 4, 9],
    'calories': [3000, 2200, 1800, 2500, 3200, 2100],
    'water': [1.0, 2.5, 1.5, 2.0, 0.8, 3.0],
    'label': [0, 1, 2, 1, 0, 2]
})
X_rf = rf_data.drop("label", axis=1)
y_rf = rf_data["label"]
rf_model = RandomForestClassifier()
rf_model.fit(X_rf, y_rf)

rf_labels = {
    0: "❗ Poor health. Focus on sleep and hydration.",
    1: "✅ Average health. Maintain your habits.",
    2: "🏆 Excellent health. Keep it up!"
}

def get_rf_insight(data):
    features = [[
        data['heart_rate'], data['steps'],
        data['sleep'], data['calories'], data['water']
    ]]
    prediction = rf_model.predict(features)[0]
    return rf_labels.get(prediction), prediction

In [ ]:
X_lstm = np.random.rand(100, 5)
y_lstm = np.random.rand(100)
X_lstm = X_lstm.reshape((100, 1, 5))

lstm_model = Sequential([
    LSTM(32, input_shape=(1, 5)),
    Dense(1)
])
lstm_model.compile(optimizer=Adam(), loss='mse')
lstm_model.fit(X_lstm, y_lstm, epochs=5, verbose=0)

def predict_health_score(data):
    features = np.array([[
        data['heart_rate'], data['steps'],
        data['sleep'], data['calories'], data['water']
    ]])
    features = features.reshape((1, 1, 5))
    score = lstm_model.predict(features, verbose=0)[0][0]
    return score

In [ ]:
X_cluster = rf_data.drop("label", axis=1)
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(X_cluster)

cluster_labels = {
    0: "Group 0: Needs improvement – low hydration/activity.",
    1: "Group 1: Balanced – moderate metrics.",
    2: "Group 2: Highly active – great habits!"
}

def get_cluster_group(data):
    features = np.array([[
        data['heart_rate'], data['steps'],
        data['sleep'], data['calories'], data['water']
    ]])
    group = kmeans.predict(features)[0]
    return cluster_labels.get(group)

In [ ]:
df = pd.read_csv("/kaggle/input/healthcare-dataset/health_data_60_days.csv")
display(df.head())

In [ ]:
def get_all_insights(row):
    data = {
        "heart_rate": row['heart_rate'],
        "steps": row['steps'],
        "sleep": row['sleep'],
        "calories": row['calories'],
        "water": row['water']
    }
    rf, rf_raw = get_rf_insight(data)
    lstm = predict_health_score(data)
    cluster = get_cluster_group(data)
    alert = "⚠️ Alert: Poor Health!" if rf_raw == 0 or lstm < 0.4 else ""
    
    return pd.Series({
        "rf_insight": rf,
        "lstm_pred": f"{lstm:.2f}",
        "cluster_group": cluster,
        "alert": alert
    })

df_insights = df.apply(get_all_insights, axis=1)
df_final = pd.concat([df, df_insights], axis=1)

display(df_final)

In [ ]:
metrics = ['heart_rate', 'steps', 'sleep', 'calories', 'water']
plt.figure(figsize=(15, 8))
for i, metric in enumerate(metrics):
    plt.subplot(2, 3, i+1)
    sns.lineplot(x=df['date'], y=df[metric])
    plt.title(metric.capitalize())
    plt.xticks(rotation=45)
    plt.tight_layout()
plt.suptitle("📈 Health Trends Over Days", fontsize=16, y=1.03)
plt.show()

In [ ]:
csv_path = "daily_health_insights.csv"
df_final.to_csv(csv_path, index=False)
print("✅ CSV file saved. Click to download:")
display(FileLink(csv_path))